This notebook can be used to load raw ultraleap data, 
save cleaned dataframes for each block, 
and generate dataframes of distances for further feature extraction

Import public packages and functions

In [2]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import compress


import openpyxl
from datetime import datetime
from scipy.spatial import distance
from scipy.ndimage import uniform_filter1d
import math
from scipy.signal import find_peaks, peak_prominences
import statistics as stat
import json
import traces

In [3]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path


In [4]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)

Import own functions

In [5]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import import_data.preprocessing_meta_info as meta_info
import sig_processing.segment_tasks as tasks
import movement_calc.helpfunctions as hp

### Saving blocks for preprocessing

Reloading own functions

In [6]:
importlib.reload(import_dat)
importlib.reload(tasks)
importlib.reload(find_paths)
importlib.reload(meta_info)
importlib.reload(hp)

<module 'movement_calc.helpfunctions' from '/Users/arianm/Documents/GitHub/ultraleap_analysis/code/movement_calc/helpfunctions.py'>

Define variables of interest

In [7]:
folder = 'patientdata'
ls_cond = ['m1', 'm0s0', 'm0s1', 'm1s0', 'm1s1']
ls_cam = ['dt', 'vr',  'st']
ls_task = ['ft', 'oc', 'ps', 'pt']
ls_side = ['left', 'right']
subs = find_paths.find_available_subs(folder)

Save blocks for feature extraction

In [169]:
for sub in subs:
    for cond in ls_cond:
        for cam in ls_cam:
            for task in ls_task:
                for side in ls_side:
                    # get defined data
                    try:
                        data = import_dat.get_data(folder = folder,
                            sub=sub, cam_pos=cam, task=task, condition=cond, side=side,
                        )

                        # get block dataframes
                        blocks = tasks.block_extraction(
                            data, sub=sub, task=task, side=side,
                            cond=cond, cam=cam, to_save=True)

                        block_path = os.path.join(
                            repo_path, 'data', 'raw_blocks', sub, task, cond, 'blocks')

                    except Exception:
                        continue

                #saving blocks as .csv files
                    for block in blocks.keys():
                        if blocks[block].empty:
                            continue
                        if not os.path.exists(block_path) and not blocks[block].empty:
                            os.makedirs(block_path)
                        blocks[block].to_csv(os.path.join(
                            block_path, f'{block}_{sub}_{cond}_{cam}_{task}_{side}.csv'))

Load blocks, calculate distances and create dist-time .csv files

In [32]:
for sub in subs:
    for cond in ls_cond:
        for task in ls_task:
            try:
                files = os.listdir(os.path.join(repo_path, 'data','raw_blocks', sub, task, cond, 'blocks'))

            except FileNotFoundError:
                continue 
            
            for file in files:
                if file == '.DS_Store':
                    continue

                # Load blocks from patients' blocks dir
                block = pd.read_csv(os.path.join(
                    repo_path, 'data', 'raw_blocks', sub, task, cond, 'blocks', file))

                if task == 'oc':
                    dist = hp.calc_distances(block, 'middle_tip', 'palm')
                elif task == 'ft':
                    dist = hp.calc_distances(block, 'index_tip', 'thumb_tip')
                elif task == 'ps':
                    dist = hp.calc_prosup_angle(block, 'thumb_tip', 'middle_tip', 'palm')
                elif task == 'pt':
                    print('Postural tremor not analyzed yet')

                dist['time'] = block.program_time.to_list()

                dist_path = os.path.join(
                    repo_path, 'data', 'distances', sub, task, cond)
                if not os.path.exists(dist_path):
                    os.makedirs(dist_path)
                dist.to_csv(os.path.join(
                    dist_path, f'dist_{file}'))

# Visualisations for self-control

In [29]:
for sub in subs:
    for cond in ls_cond:
        for task in ls_task:
            try:
                files = os.listdir(os.path.join(repo_path, 'data','raw_blocks', sub, task, cond, 'dist'))

            except FileNotFoundError:
                continue 
            
            for file in files:
                if file == '.DS_Store':
                    continue

                # Load blocks from patients' blocks dir
                block = pd.read_csv(os.path.join(
                    repo_path, 'data', 'raw_blocks', sub, task, cond, 'dist', file), index_col= 0)

                plt.figure(figsize=(18, 10))
                plt.plot(block[])
                plt.plot(block.iloc[].iloc[:,0], 'o', color= 'red')
                plt.plot(block.iloc[].iloc[:,0], 'o', color= 'blue')    
                plt.show()    
                            

SyntaxError: invalid syntax (1055887893.py, line 19)

In [1085]:
def new(dist_dataframe):
    '''
    Try out further and adapt for PS
    '''

    fps = hp.calc_fps(dist_dataframe)

    dist_array = np.array(dist_dataframe.iloc[:,0])

    ...

    plt.figure(figsize=(18, 10))
    plt.plot(dist_array)
    plt.plot(dist_dataframe.iloc[ ].iloc[:,0], 'o', color= 'red')
    plt.plot(dist_dataframe.iloc[ ].iloc[:,0], 'o', color= 'blue')
    # plt.show()
    return